In [1]:
include("CVaR_functions.jl")
using .CVaR_functions
using LinearAlgebra
using Statistics
using SparseArrays

In [3]:
using CSV
using DataFrames

# Load the CSV file into a DataFrame
df = CSV.read("simulations_data.csv", DataFrame)

# Convert the DataFrame to a matrix
matrix_data = Matrix(df)


10000×8 Matrix{Float64}:
  0.0574226    0.0335883  0.168124    …  -0.0183215   -0.00640398
 -0.020569    -0.0220066  0.0639414      -0.0232601    0.0166911
  0.0698219    0.0591938  0.04715         0.0264266    0.0169535
  0.0102428    0.113952   0.052645       -0.0243514   -0.0523726
 -0.0171983    0.0461375  0.0643485      -0.0215266    0.00160292
  0.00622201   0.0930191  0.0300492   …  -0.0665092    0.0348673
  0.0687899   -0.0406231  0.0465296      -0.0289802    0.0822492
  0.0226579    0.116673   0.0361669       0.0381087    0.0411748
  0.0471813    0.0209458  0.0320813       0.0184561   -0.0157252
  0.0543028    0.0917352  0.00200951      0.0131739   -0.0588187
  ⋮                                   ⋱               
  0.0818182    0.0260692  0.0833563      -0.0224092   -0.0548706
  0.0331751   -0.0267028  0.10099        -0.0313692    0.0153552
  0.017132     0.0671928  0.050697       -0.034256    -0.0240901
  0.0199469    0.0447889  0.0851107      -0.040453    -0.00887618
 -0.036

In [14]:
current_weights = vcat(zeros(5),[0.4,0.6],zeros(9))
new_weights, updates  = update_portfolio_weights_2(current_weights,-matrix_data[1:10000,1:8],0.01,9999,true,0.05)


weights_new minus costs must sum to
0.0


([0.0, 0.0, 0.14448069703621685, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.1074226382805037 -0.08358830310501511 … 0.0 0.0; -0.0294309721252653 -0.0279933893458635 … 0.0 0.0; … ; 0.04145210553579541 0.05354265665220357 … 0.0 0.0; 1.05 1.05 … 0.0 0.0], [-0.008930887293219704, -0.005435049836597362, -0.03643155107974022, -0.046613528659587365, -0.006955384378190941, 0.019618828686148317, 0.013193564206467761, -0.00376426862310876, -0.03732486085246048, 0.11269181102510016  …  -0.013641815330498085, -0.03706492125311697, 0.04155338861558992, -0.06498095125363534, -0.0008726359906964564, -0.011522029220362141, 0.01833097715466066, -0.03786202855787298, 0.01, 0.0])

In [25]:
using HTTP
using JSON
using Dates
using DataFrames

function download_stock_data(tickers::Vector{String}, api_key::String)
    base_url = "https://www.alphavantage.co/query"
    
    function fetch_data(ticker)
        url = "$base_url?function=TIME_SERIES_DAILY_ADJUSTED&symbol=$ticker&apikey=$api_key"
        response = HTTP.get(url)
        if response.status != 200
            println("Error fetching data for $ticker: $(response.status)")
            return nothing
        end
        return JSON.parse(String(response.body))
    end

    data = Dict{Any, Any}()
    for ticker in tickers
        println("Fetching data for $ticker...")
        stock_data = fetch_data(ticker)
        if stock_data != nothing && haskey(stock_data, "Time Series (Daily)") 
            data[ticker] = stock_data["Time Series (Daily)"]
        end
    end

    return data
end

function create_dataframe(stock_data::Dict{Any, Any})
    df_list = []
    
    for (ticker, daily_data) in stock_data
        dates = Date[]
        adj_closes = Float64[]
        
        for (date_str, price_data) in daily_data
            date = Date(date_str, "yyyy-mm-dd")
            adj_close = parse(Float64, price_data["5. adjusted close"])
            push!(dates, date)
            push!(adj_closes, adj_close)
        end
        
        df = DataFrame(Date=dates, Adjusted_Close=adj_closes)
        df.Ticker = ticker
        push!(df_list, df)
    end
    
    combined_df = vcat(df_list)
    return combined_df
end

# Example usage:
tickers = ["AAPL", "GOOGL", "MSFT"]
api_key = "VWLXYQW5UFLW7OQX"
stock_data = download_stock_data(tickers, api_key)
df = create_dataframe(stock_data)
println(df)


Fetching data for AAPL...
Fetching data for GOOGL...
Fetching data for MSFT...
Any[]


In [31]:
AlphaVantage.GLOBAL[]

UndefVarError: UndefVarError: `GLOBAL` not defined

In [43]:
ENV["ALPHA_VANTAGE_API_KEY"] = "VWLXYQW5UFLW7OQX"


using AlphaVantage
using DataFrames, StatsPlots, Dates


ArgumentError: ArgumentError: column name :timestamp not found in the data frame

In [56]:
using DataFramesMeta
using Plots

In [61]:
function raw_to_dataframe(rawData)
    df = DataFrame(rawData[1],:auto)
    dfNames = Symbol.(vcat(rawData[2]...))
    df = rename(df, dfNames)
    println(df)
    df.Date = Date.(df.timestamp)
    for x in (:open, :high, :low, :close, :adjusted_close, :dividend_amount)
        df[!, x] = Float64.(df[!, x])
    end 
    df.volume = Int64.(df.volume)
    return df
end

function intra_to_dataframe(rawData)
    df = DataFrame(rawData[1],:auto)
    dfNames = Symbol.(vcat(rawData[2]...))
    df = rename(df, dfNames)

    df.DateTime = DateTime.(df.timestamp, "yyyy-mm-dd HH:MM:SS")
    for x in (:open, :high, :low, :close)
        df[!, x] = Float64.(df[!, x])
    end 
    df.volume = Int64.(df.volume)
    return df
end

intra_to_dataframe (generic function with 1 method)

In [62]:
tslaRaw = AlphaVantage.time_series_daily_adjusted("TSLA", outputsize="full", datatype="csv")
tsla = raw_to_dataframe(tslaRaw);
first(tsla, 5)


2×1 DataFrame
 Row │ {                                 
     │ Any                               
─────┼───────────────────────────────────
   1 │     "Information": "Thank you fo…
   2 │ }


ArgumentError: ArgumentError: column name :timestamp not found in the data frame